<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/BART_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# fairseqインストール(gitから)
!git clone https://github.com/utanaka2000/fairseq.git
%cd fairseq
!git fetch origin
!git checkout japanese_bart_pretrained_model
!git branch

In [ ]:
# インストール
! pip install .

In [ ]:
# fairseq有効化
!echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] = "/env/python"
os.environ['PYTHONPATH'] += ":/content/fairseq/"

!echo $PYTHONPATH

/env/python
/env/python:/content/fairseq/


In [ ]:
# インストールの確認
!pip show fairseq

In [ ]:
%cd /content
!wget http://lotus.kuee.kyoto-u.ac.jp/nl-resource/JapaneseBARTPretrainedModel/japanese_bart_base_1.1.tar.gz
!tar -zxvf japanese_bart_base_1.1.tar.gz

In [ ]:
! ls japanese_bart_base_1.1

bart_model.pt  dict.txt  sp.model


####  Juman++

In [ ]:
# jumanpp-2.0.0-rc3 download
%cd /content
!wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc3/jumanpp-2.0.0-rc3.tar.xz

In [ ]:
# unzip a file
!tar xvf jumanpp-2.0.0-rc3.tar.xz

In [ ]:
## jumanpp-2.0.0-rc3/src/core/analysis/analyzer.h
## 16,17行目
#  size_t pageSize = 48 * 1024 * 1024;
#  size_t maxInputBytes = 48 * 1024;

%%bash
sed -i -e "16c size_t pageSize = 48 * 1024 * 1024;" jumanpp-2.0.0-rc3/src/core/analysis/analyzer.h
sed -i -e "17c size_t maxInputBytes = 48 * 1024;" jumanpp-2.0.0-rc3/src/core/analysis/analyzer.h

! head -17 jumanpp-2.0.0-rc3/src/core/analysis/analyzer.h | tail -2

size_t pageSize = 48 * 1024 * 1024;
size_t maxInputBytes = 48 * 1024;


In [ ]:
## jumanpp-2.0.0-rc3/src/core/input/stream_reader.h
## 27,28行目
#  u64 maxInputLength_ = 48 * 1024;
#  u64 maxCommentLength_ = 48 * 1024;

%%bash
sed -i -e "27c u64 maxInputLength_ = 48 * 1024;" jumanpp-2.0.0-rc3/src/core/input/stream_reader.h
sed -i -e "28c u64 maxCommentLength_ = 48 * 1024;" jumanpp-2.0.0-rc3/src/core/input/stream_reader.h

! head -28 jumanpp-2.0.0-rc3/src/core/input/stream_reader.h | tail -2

u64 maxInputLength_ = 48 * 1024;
u64 maxCommentLength_ = 48 * 1024;


In [ ]:
# build jumanpp
%cd jumanpp-2.0.0-rc3/
!mkdir build

/content/jumanpp-2.0.0-rc3


In [15]:
!cmake . -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
!make

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
2.0.0-rc3
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Could NOT find Protobuf (missing: Protobuf_LIBRARIES Protobuf_INCLUDE_DIR) 
Detecting protobuf... FALSE
protobuf library

In [16]:
# install jumanpp
!sudo make install

Consolidate compiler generated dependencies of target jpp_util
[  6%] Built target jpp_util
Consolidate compiler generated dependencies of target jpp_test_catch_main_lib
[  7%] Built target jpp_test_catch_main_lib
Consolidate compiler generated dependencies of target jpp_util_test
[ 16%] Built target jpp_util_test
Consolidate compiler generated dependencies of target testing_infra_test
[ 17%] Built target testing_infra_test
Consolidate compiler generated dependencies of target jpp_rnn
[ 18%] Built target jpp_rnn
Consolidate compiler generated dependencies of target pathie
[ 21%] Built target pathie
Consolidate compiler generated dependencies of target jpp_core
[ 49%] Built target jpp_core
Consolidate compiler generated dependencies of target jpp_core_train
[ 54%] Built target jpp_core_train
Consolidate compiler generated dependencies of target jpp_core_tests
[ 61%] Built target jpp_core_tests
Consolidate compiler generated dependencies of target jpp_core_analysis_tests
[ 68%] Built tar

In [17]:
!echo "本日は晴天なり" | jumanpp

本日 ほんじつ 本日 名詞 6 時相名詞 10 * 0 * 0 "代表表記:本日/ほんじつ カテゴリ:時間"
は は は 助詞 9 副助詞 2 * 0 * 0 NIL
晴天 せいてん 晴天 名詞 6 普通名詞 1 * 0 * 0 "代表表記:晴天/せいてん カテゴリ:抽象物"
なり なり なり 助詞 9 接続助詞 3 * 0 * 0 NIL
EOS



#データの準備

bart_datasets.csv をアップロードする

In [ ]:
%cd /content

# アップロードが遅いので…
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## 既に、Google　Drive　に準備済みのデータがあれば、それをコピー

!cp /content/drive/MyDrive/bart/dict.src.txt　./
!cp /content/drive/MyDrive/bart/dict.tgt.txt　./
!cp /content/drive/MyDrive/bart/valid.src-tgt.src　./
!cp /content/drive/MyDrive/bart/valid.src-tgt.tgt　./
!cp /content/drive/MyDrive/bart/test.src-tgt.src　./
!cp /content/drive/MyDrive/bart/test.src-tgt.tgt　./
!cp /content/drive/MyDrive/bart/train.src-tgt.src　./
!cp /content/drive/MyDrive/bart/train.src-tgt.tgt　./

In [ ]:
!cp /content/drive/MyDrive/bart/bart_datasets.csv ./

In [ ]:
!ls -alh bart_datasets.csv

-rw------- 1 root root 153M Aug  8 10:53 bart_datasets.csv


In [ ]:
!head bart_datasets.csv

In [ ]:
import pandas as pd

livedoor_df = pd.read_csv("bart_datasets.csv", sep=',', encoding='UTF-8')
input_df = livedoor_df.copy()

input_df["tgt_length"] = input_df["tgt"].apply(lambda x: len(str(x)))
input_df["src_length"] = input_df["src"].apply(lambda x: len(str(x)))
input_df["rate"] = input_df["tgt_length"] / input_df["src_length"]

mask = input_df['src_length'] <= 1500
input_df = input_df[mask]

mask = input_df['rate'] <= 0.5
input_df = input_df[mask]

mask = input_df['rate'] >= 0.05
input_df = input_df[mask]

input_df = input_df.sample(frac=1)

In [ ]:
input_df

In [ ]:
len(input_df)

46267

In [ ]:
train_df = input_df[:42000]
val_df = input_df[42000:44000]
test_df = input_df[44000:]

train_src = train_df["src"].to_list()
train_tgt = train_df["tgt"].to_list()
val_src = val_df["src"].to_list()
val_tgt = val_df["tgt"].to_list()
test_src = test_df["src"].to_list()
test_tgt = test_df["tgt"].to_list()

# 記号や特殊文字を削除する関数
def delete_special_character(str_list):
    new_list = []
    characters = "■◆☆●\n"
    for strength in str_list:
        for x in range(len(characters)):
            strength = str(strength).replace(characters[x],"")
            new_list.append(strength)
    return new_list

train_src = delete_special_character(train_src)
train_tgt = delete_special_character(train_tgt)
val_src = delete_special_character(val_src)
val_tgt = delete_special_character(val_tgt)
test_src = delete_special_character(test_src)
test_tgt = delete_special_character(test_tgt)

with open("train_src.txt", 'w') as f:
  for d in train_src:
    f.write("%s\n" % d)  

with open("train_tgt.txt", 'w') as f:
  for d in train_tgt:
    f.write("%s\n" % d)  

with open("val_src.txt", 'w') as f:
  for d in val_src:
    f.write("%s\n" % d)  

with open("val_tgt.txt", 'w') as f:
  for d in val_tgt:
    f.write("%s\n" % d)

with open("test_src.txt", 'w') as f:
  for d in test_src:
    f.write("%s\n" % d)  

with open("test_tgt.txt", 'w') as f:
  for d in test_tgt:
    f.write("%s\n" % d) 

In [ ]:
!pwd
!ls

/content/jumanpp-2.0.0-rc3
bart_datasets.csv    CONTRIBUTORS	   README.md	  val_src.txt
build		     CTestTestfile.cmake   src		  val_tgt.txt
cmake		     docs		   test		  version
CMakeCache.txt	     install_manifest.txt  test_src.txt   version.cmake
CMakeFiles	     libs		   test_tgt.txt
cmake_install.cmake  Makefile		   train_src.txt
CMakeLists.txt	     model		   train_tgt.txt



##前処理用の環境変数の設定

In [ ]:
# データセットの前処理の設定
%env TRAIN_SRC=train_src.txt
%env TRAIN_TGT=train_tgt.txt
%env VALID_SRC=val_src.txt
%env VALID_TGT=val_tgt.txt
%env TEST_SRC=test_src.txt
%env TEST_TGT=test_tgt.txt
# ダウンロードしたセンテンスピースモデル
%env SENTENCEPIECE_MODEL=japanese_bart_base_1.1/sp.model
# 前処理後のファイルを入れるフォルダ
%env DATASET_DIR=datasets/
# ダウンロードした辞書ファイル
%env DICT=japanese_bart_base_1.1/dict.txt


##前処理実行

In [ ]:
%cd /content

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cat $TRAIN_SRC | python3 fairseq/jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/train.src-tgt.src
!cp $DATASET_DIR/train.src-tgt.src /content/drive/MyDrive/bart/
!cat $TRAIN_TGT | python3 fairseq/jaBART_preprocess.py  --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/train.src-tgt.tgt
!cp $DATASET_DIR/train.src-tgt.tgt /content/drive/MyDrive/bart/
!cat $VALID_SRC | python3 fairseq/jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/valid.src-tgt.src
!cp $DATASET_DIR/valid.src-tgt.src /content/drive/MyDrive/bart/
!cat $VALID_TGT | python3 fairseq/jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/valid.src-tgt.tgt
!cp $DATASET_DIR/valid.src-tgt.tgt /content/drive/MyDrive/bart/
!cat $TEST_SRC | python3 fairseq/jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/test.src-tgt.src
!cp $DATASET_DIR/test.src-tgt.src /content/drive/MyDrive/bart/
!cat $TEST_TGT | python3 fairseq/jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/test.src-tgt.tgt
!cp $DATASET_DIR/test.src-tgt.tgt /content/drive/MyDrive/bart/
!cp $DICT $DATASET_DIR/dict.src.txt
!cp $DATASET_DIR/dict.src.txt /content/drive/MyDrive/bart/
!cp $DICT $DATASET_DIR/dict.tgt.txt
!cp $DATASET_DIR/dict.tgt.txt /content/drive/MyDrive/bart/


##ファインチューニング


##ファインチューニング用の環境変数の設定


In [ ]:
# ダウンロードした事前学習済みBARTモデル
%env PRETRAINED_MODEL=japanese_bart_base_1.1/bart_model.pt
# bart_baseまたはbart_largeを設定
%env BART=bart_base
# その他保存用フォルダの設定
%env TENSORBOARD_DIR=log/
%env SAVE_MODEL_DIR=save/
%env RESULT=result.txt


##ファインチューニング実行

In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train $DATASET_DIR --arch $BART --restore-file $PRETRAINED_MODEL \
--save-dir $SAVE_MODEL_DIR --tensorboard-logdir $TENSORBOARD_DIR \
--task translation_from_pretrained_bart --source-lang src --target-lang tgt \
--criterion label_smoothed_cross_entropy --label-smoothing 0.2 --dataset-impl raw \
--optimizer adam --adam-eps 1e-06 --adam-betas '{0.9, 0.98}' --lr-scheduler polynomial_decay --lr 3e-05 --min-lr -1 \
--warmup-updates 2500 --total-num-update 1000 --dropout 0.3 --attention-dropout 0.1  --weight-decay 0.0 \
--max-tokens 1024 --update-freq 5 --save-interval -1 --no-epoch-checkpoints --seed 222 --log-format simple --log-interval 2 \
--reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler  --save-interval-updates 10000 \
--ddp-backend no_c10d --max-epoch 5 \
--encoder-normalize-before --decoder-normalize-before --prepend-bos \
--skip-invalid-size-inputs-valid-test


##ファインチューニング後モデルの保存

In [ ]:
%env MODEL_NAME=livedoor_5_eopch

!mkdir models/$MODEL_NAME

!cp -rf $SAVE_MODEL_DIR models/$MODEL_NAME
!cp -rf $DICT models/$MODEL_NAME
!cp -rf $SENTENCEPIECE_MODEL models/$MODEL_NAME
!cp -rf $TENSORBOARD_DIR models/$MODEL_NAME
!cp -rf $DATASET_DIR models/$MODEL_NAME


##要約実行


##要約用スクリプト

In [ ]:
from collections import namedtuple
import fileinput
import logging
import math
import sys
import time
import os
import numpy as np
import torch
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from fairseq.data import encoders
from fairseq.token_generation_constraints import pack_constraints, unpack_constraints
import zenhan
from pyknp import Juman
import sentencepiece

logging.basicConfig(
    format='%(name)s | %(message)s',
    level=os.environ.get('LOGLEVEL', 'INFO').upper(),
    stream=sys.stdout,
)
logger = logging.getLogger('infer')

Batch = namedtuple('Batch', 'ids src_tokens src_lengths constraints')
Translation = namedtuple('Translation', 'src_str hypos pos_scores alignments')


def get_symbols_to_strip_from_output(generator):
    if hasattr(generator, 'symbols_to_strip_from_output'):
        return generator.symbols_to_strip_from_output
    else:
        return {generator.eos}


def buffered_read(input, buffer_size):
    buffer = []
    with fileinput.input(files=[input], openhook=fileinput.hook_encoded("utf-8")) as h:
        for src_str in h:
            buffer.append(src_str.strip())
            if len(buffer) >= buffer_size:
                yield buffer
                buffer = []

    if len(buffer) > 0:
        yield buffer


def make_batches(lines, args, task, max_positions, encode_fn):
    tokens = [
        task.source_dictionary.encode_line(
            encode_fn(src_str), add_if_not_exist=False
        ).long()
        for src_str in lines
    ]

    if args.constraints:
        constraints_tensor = pack_constraints(batch_constraints)
    else:
        constraints_tensor = None
    lengths = [t.numel() for t in tokens]
    itr = task.get_batch_iterator(
        dataset=task.build_dataset_for_inference(tokens, lengths, constraints=constraints_tensor),
        max_tokens=args.max_tokens,
        max_sentences=args.max_sentences,
        max_positions=max_positions,
        ignore_invalid_inputs=args.skip_invalid_size_inputs_valid_test
    ).next_epoch_itr(shuffle=False)
    for batch in itr:
        ids = batch['id']
        src_tokens = batch['net_input']['src_tokens']
        src_lengths = batch['net_input']['src_lengths']
        constraints = batch.get("constraints", None)

        yield Batch(
            ids=ids,
            src_tokens=src_tokens,
            src_lengths=src_lengths,
            constraints=constraints,
        )


def main(args):
    start_time = time.time()
    total_translate_time = 0

    utils.import_user_module(args)

    if args.buffer_size < 1:
        args.buffer_size = 1
    if args.max_tokens is None and args.max_sentences is None:
        args.max_sentences = 1

    assert not args.sampling or args.nbest == args.beam, \
        '--sampling requires --nbest to be equal to --beam'
    assert not args.max_sentences or args.max_sentences <= args.buffer_size, \
        '--max-sentences/--batch-size cannot be larger than --buffer-size'

    # Fix seed for stochastic decoding
    if args.seed is not None and not args.no_seed_provided:
        np.random.seed(args.seed)
        utils.set_torch_seed(args.seed)

    use_cuda = False

    # Setup task, e.g., translation
    task = tasks.setup_task(args)

    # Load ensemble
    logger.info('loading model(s) from {}'.format(args.path))
    models, _model_args = checkpoint_utils.load_model_ensemble(
        args.pa。rrides=eval(args.model_overrides),
        task=task,
        suffix=getattr(args, "checkpoint_suffix", ""),
    )

    # Set dictionaries
    src_dict = task.source_dictionary
    tgt_dict = task.target_dictionary

    # Optimize ensemble for generation
    for model in models:
        model.prepare_for_inference_(args)
        if args.fp16:
            model.half()
        if use_cuda:
            model.cuda()

    # Initialize generator
    generator = task.build_generator(models, args)

    # Handle tokenization and BPE
    tokenizer = encoders.build_tokenizer(args)
    bpe = encoders.build_bpe(args)

    jumanpp = Juman()
    spm = sentencepiece.SentencePieceProcessor()
    spm.Load(args.bpe_model)
       return ' '.join([mrph.midasi for mrph in result.mrph_list()])

    def bpe_encode(line, spm):
        return ' '.join(spm.EncodeAsPieces(line.strip()))

    def encode_fn(x):
        x = x.strip()
        x = zenhan.h2z(x)
        x = juman_split(x, jumanpp)
        x = bpe_encode(x, spm)
        return x

    def decode_fn(x):
        x = x.translate({ord(i): None for i in ['▁', ' ']})
        return x

    align_dict = utils.load_align_dict(args.replace_unk)

    max_positions = utils.resolve_max_positions(
        task.max_positions(),
        *[model.max_positions() for model in models]
    )

    if args.constraints:
        logger.warning("NOTE: Constrained decoding currently assumes a shared subword vocabulary.")

    if args.buffer_size > 1:
        logger.info('Sentence buffer size: %s', args.buffer_size)
    logger.info('NOTE: hypothesis and token scores are output in base 2')
    logger.info('Type the input sentence and press return:')
    start_id = 0

    # 入力用ファイルを指定する
    input_text = 'test_src.txt'

    # 出力用の配列
    output_texts = []
    output_texts_2 = []
    output_texts_3 = []

    for inputs in buffered_read(input_text, args.buffer_size):
        results = []
        for batch in make_batches(inputs, args, task, max_positions, encode_fn):
            bsz = batch.src_tokens.size(0)
            src_tokens = batch.src_tokens
            src_lengths = batch.src_lengths
            constraints = batch.constraints
            if use_cuda:
                src_tokens = src_tokens.cuda()
                src_lengths = src_lengths.cuda()
                if constraints is not None:
                    constraints = constraints.cuda()

            sample = {
                'net_input': {
                    'src_tokens': src_tokens,
                    'src_lengths': src_lengths,
                },
            }
            translate_start_time = time.time()
            translations = task.inference_step(generator, models, sample, constraints=constraints)
            translate_time = time.time() - translate_start_time
            total_translate_time += translate_time
            list_constraints = [[] for _ in range(bsz)]
            if args.constraints:
                list_constraints = [unpack_constraints(c) for c in constraints]
            for i, (id, hypos) in enumerate(zip(batch.ids.tolist(), translations)):
                src_tokens_i = utils.strip_pad(src_tokens[i], tgt_dict.pad())
                constraints = list_constraints[i]
                results.append((start_id + id, src_tokens_i, hypos,
                    {
                        "constraints": constraints,
                        "time": translate_time / len(translations)
                    }))

        # sort output to match input order
        for id_, src_tokens, hypos, info in sorted(results, key=lambda x: x[0]):
            if src_dict is not None:
                src_str = src_dict.string(src_tokens, args.remove_bpe)
                print(f'Inference time: {info["time"]:.3f} seconds')

            # Process top predictions
            for hypo_i, hypo in enumerate(hypos[:min(len(hypos), args.nbest)]):
                hypo_tokens, hypo_str, alignment = utils.post_process_prediction(
                    hypo_tokens=hypo['tokens'].int().cpu(),
                    src_str=src_str,
                    alignment=hypo['alignment'],
                    align_dict=align_dict,
                    tgt_dict=tgt_dict,
                    remove_bpe=args.remove_bpe,
                    extra_symbols_to_ignore=get_symbols_to_strip_from_output(generator),
                )
                detok_hypo_str = decode_fn(hypo_str)
                score = hypo['score'] / math.log(2)  # convert to base 2

                if hypo_i == 0:
                    output_texts.append(detok_hypo_str)
                if hypo_i == 1:
                    output_texts_2.append(detok_hypo_str)
                if hypo_i == 2:
                    output_texts_3.append(detok_hypo_str)

                print(f'Top {hypo_i+1} prediction score: {score}')

        # update running id_ counter
        start_id += len(inputs)
    
    # 要約結果を１〜３番目までそれぞれ書き出し
    with open(f"{SAVE_MODEL_NAME}_test_tgt.txt", 'w') as f:
        count = 0
        for d in output_texts:
            count+=1
            f.write("%s\n" % d)
    with open(f"{SAVE_MODEL_NAME}_test_tgt_2.txt", 'w') as f:
        count = 0
        for d in output_texts_2:
            count+=1
            f.write("%s\n" % d)
    with open(f"{SAVE_MODEL_NAME}_test_tgt_3.txt", 'w') as f:
        count = 0
        for d in output_texts_3:
            count+=1
            f.write("%s\n" % d)


##要約用スクリプトの実行

In [ ]:
SAVE_MODEL_NAME = "livedoor_5_eopch"
MODEL_NAME = "models/" + SAVE_MODEL_NAME

def cli_main():
    parser = options.get_interactive_generation_parser()
    parser.add_argument('--bpe_model', default='', required=True)
    parser.add_argument('--bpe_dict', default='', required=True)

    bpe_model = MODEL_NAME + "/sp.model"
    bpe_dict = MODEL_NAME + "/dict.txt"
    datasets_dir = MODEL_NAME + "/datasets"
    tuning_model = MODEL_NAME + "/save/checkpoint_best.pt"

    input_args = [
        datasets_dir, 
        "--path", tuning_model,
        "--task", "translation_from_pretrained_bart",
        "--max-sentences", "1",
        "--bpe_model", bpe_model,
        "--bpe_dict", bpe_dict,
        "--nbest", "3",
        "--skip-invalid-size-inputs-valid-test"
    ]
    args = options.parse_args_and_arch(parser,input_args)

    distributed_utils.call_main(args, main)


cli_main()